## Step 1: Install Dependencies

In [13]:
!pip install pdfplumber -q
print("Dependencies installed successfully!")

Dependencies installed successfully!


## Step 2: Create Sample FNOL Documents

In [14]:
# Create sample FNOL document 1 - Fast-track case
sample_1 = """FIRST NOTICE OF LOSS (FNOL)
================================

POLICY INFORMATION
------------------
Policy Number: AUTO-2024-78901
Policyholder Name: Sarah Johnson
Effective From: 01/15/2024
Effective To: 01/15/2025

INCIDENT INFORMATION
--------------------
Incident Date: 02/03/2026
Incident Time: 3:45 PM
Location: 123 Main Street, Springfield, IL
Description: Minor fender bender in parking lot. The insured vehicle was backing out of a parking space when it made contact with a stationary shopping cart. Minor dent and scratches on rear bumper. No other vehicles involved. No injuries reported.

INVOLVED PARTIES
----------------
Claimant Name: Sarah Johnson
Contact Phone: (555) 234-5678
Contact Email: sarah.johnson@email.com

Third Party: None

ASSET DETAILS
-------------
Asset Type: Vehicle
Asset ID: VIN-1HGBH41JXMN109186
Estimated Damage: $1,200.00

OTHER INFORMATION
-----------------
Claim Type: Property Damage
Attachments: Photos of damage (3 files), Parking lot diagram
Initial Estimate: $1,200.00
"""

# Create sample FNOL document 2 - Injury case
sample_2 = """FIRST NOTICE OF LOSS (FNOL)
================================

POLICY INFORMATION
------------------
Policy Number: AUTO-2024-55432
Policyholder Name: Michael Chen
Effective From: 06/10/2024
Effective To: 06/10/2025

INCIDENT INFORMATION
--------------------
Incident Date: 02/01/2026
Incident Time: 8:20 AM
Location: Highway 101 Northbound, Mile Marker 45
Description: Two-vehicle collision on highway during morning commute. The insured vehicle was rear-ended by another vehicle while stopped in traffic. Both drivers complained of neck pain. Police report filed at scene. Airbags deployed in both vehicles.

INVOLVED PARTIES
----------------
Claimant Name: Michael Chen
Contact Phone: (555) 876-5432
Contact Email: mchen@email.com

Third Party Name: Jennifer Martinez
Third Party Phone: (555) 234-9876

ASSET DETAILS
-------------
Asset Type: Vehicle
Asset ID: VIN-2HGFG12848H543210
Estimated Damage: $18,500.00

OTHER INFORMATION
-----------------
Claim Type: Personal Injury
Attachments: Police report, Photos, Medical records (ER visit)
Initial Estimate: $18,500.00 (vehicle only, medical costs TBD)
"""

# Create sample FNOL document 3 - Fraud case
sample_3 = """FIRST NOTICE OF LOSS (FNOL)
================================

POLICY INFORMATION
------------------
Policy Number: AUTO-2025-99234
Policyholder Name: Robert Williams
Effective From: 01/20/2025
Effective To: 01/20/2026

INCIDENT INFORMATION
--------------------
Incident Date: 01/28/2026
Incident Time: 11:30 PM
Location: Remote parking lot on Oak Street
Description: The insured claims that vehicle was damaged while parked overnight. However, witness statements are inconsistent with the reported timeline. Damage pattern suggests possible staged incident. No surveillance footage available from the claimed location.

INVOLVED PARTIES
----------------
Claimant Name: Robert Williams
Contact Phone: (555) 999-1234
Contact Email: rwilliams@email.com

Third Party: Unknown (Hit and run reported)

ASSET DETAILS
-------------
Asset Type: Vehicle
Asset ID: VIN-3FADP4EJ1DM654321
Estimated Damage: $8,900.00

OTHER INFORMATION
-----------------
Claim Type: Property Damage
Attachments: Photos of damage (limited), Witness statement (conflicting)
Initial Estimate: $8,900.00
"""

# Save to files
with open('sample_fnol_1.txt', 'w') as f:
    f.write(sample_1)

with open('sample_fnol_2.txt', 'w') as f:
    f.write(sample_2)

with open('sample_fnol_3.txt', 'w') as f:
    f.write(sample_3)

print("Sample FNOL documents created successfully!")
print("   - sample_fnol_1.txt (Fast-track case)")
print("   - sample_fnol_2.txt (Injury case)")
print("   - sample_fnol_3.txt (Fraud investigation case)")

Sample FNOL documents created successfully!
   - sample_fnol_1.txt (Fast-track case)
   - sample_fnol_2.txt (Injury case)
   - sample_fnol_3.txt (Fraud investigation case)


## Step 3: Implement the Claims Processing Agent

In [15]:
import json
import re
from typing import Dict, List, Any
from pathlib import Path

class ClaimsProcessingAgent:
    """Agent for processing First Notice of Loss (FNOL) documents"""

    # Define mandatory fields
    MANDATORY_FIELDS = [
        'policy_number', 'policyholder_name', 'incident_date',
        'incident_location', 'claim_type', 'estimated_damage'
    ]

    # Routing thresholds and keywords
    FAST_TRACK_THRESHOLD = 25000
    FRAUD_KEYWORDS = ['fraud', 'inconsistent', 'staged', 'suspicious', 'fabricated']
    INJURY_CLAIM_TYPES = ['injury', 'personal injury', 'bodily injury', 'medical']

    def extract_from_txt(self, txt_path: str) -> str:
        """Extract text from TXT file"""
        try:
            with open(txt_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            print(f"Error reading TXT: {e}")
            return ""

    def extract_fields(self, text: str) -> Dict[str, Any]:
        """Extract key fields from FNOL document text using pattern matching"""
        fields = {}

        # Policy Information
        policy_number = re.search(r'policy\s*(?:number|#|no\.?)[\s:]*([A-Z0-9\-]+)', text, re.IGNORECASE)
        if policy_number:
            fields['policy_number'] = policy_number.group(1).strip()

        policyholder = re.search(r'policyholder\s*(?:name)?[\s:]*([A-Za-z\s]+?)(?:\n|Date|Policy)', text, re.IGNORECASE)
        if policyholder:
            fields['policyholder_name'] = policyholder.group(1).strip()

        # Incident Information
        incident_date = re.search(r'(?:incident|accident|loss)\s*date[\s:]*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})', text, re.IGNORECASE)
        if incident_date:
            fields['incident_date'] = incident_date.group(1).strip()

        incident_time = re.search(r'(?:incident|accident|loss)\s*time[\s:]*(\d{1,2}:\d{2}\s*(?:AM|PM)?)', text, re.IGNORECASE)
        if incident_time:
            fields['incident_time'] = incident_time.group(1).strip()

        location = re.search(r'(?:incident\s*)?location[\s:]*([^\n]+)', text, re.IGNORECASE)
        if location:
            fields['incident_location'] = location.group(1).strip()

        description = re.search(r'(?:incident\s*)?description[\s:]*([^\n]+(?:\n(?!\w+:)[^\n]+)*)', text, re.IGNORECASE)
        if description:
            fields['incident_description'] = description.group(1).strip()

        # Involved Parties
        claimant = re.search(r'claimant\s*(?:name)?[\s:]*([A-Za-z\s]+?)(?:\n|Contact)', text, re.IGNORECASE)
        if claimant:
            fields['claimant_name'] = claimant.group(1).strip()

        # Contact details
        phone = re.search(r'(?:phone|contact|tel)[\s:]*(\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', text, re.IGNORECASE)
        if phone:
            fields['contact_phone'] = phone.group(1).strip()

        email = re.search(r'email[\s:]*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', text, re.IGNORECASE)
        if email:
            fields['contact_email'] = email.group(1).strip()

        # Asset Details
        asset_type = re.search(r'asset\s*type[\s:]*([^\n]+)', text, re.IGNORECASE)
        if asset_type:
            fields['asset_type'] = asset_type.group(1).strip()

        asset_id = re.search(r'asset\s*(?:id|number)[\s:]*([A-Z0-9\-]+)', text, re.IGNORECASE)
        if asset_id:
            fields['asset_id'] = asset_id.group(1).strip()

        # Estimated damage
        damage_patterns = [
            r'estimated\s*damage[\s:]*\$?([\d,]+(?:\.\d{2})?)',
            r'damage\s*estimate[\s:]*\$?([\d,]+(?:\.\d{2})?)',
            r'initial\s*estimate[\s:]*\$?([\d,]+(?:\.\d{2})?)',
        ]

        for pattern in damage_patterns:
            damage = re.search(pattern, text, re.IGNORECASE)
            if damage:
                damage_value = damage.group(1).replace(',', '')
                fields['estimated_damage'] = float(damage_value)
                break

        # Claim Type
        claim_type = re.search(r'claim\s*type[\s:]*([^\n]+)', text, re.IGNORECASE)
        if claim_type:
            fields['claim_type'] = claim_type.group(1).strip()

        return fields

    def identify_missing_fields(self, extracted_fields: Dict[str, Any]) -> List[str]:
        """Identify which mandatory fields are missing"""
        missing = []
        for field in self.MANDATORY_FIELDS:
            if field not in extracted_fields or not extracted_fields[field]:
                missing.append(field)
        return missing

    def route_claim(self, extracted_fields: Dict[str, Any], missing_fields: List[str]) -> tuple:
        """Route claim based on business rules"""
        reasons = []

        # Rule 1: Missing mandatory fields → Manual review
        if missing_fields:
            route = "Manual Review"
            reasons.append(f"Missing mandatory fields: {', '.join(missing_fields)}")
            return route, " | ".join(reasons)

        # Rule 2: Fraud detection → Investigation Flag
        description = extracted_fields.get('incident_description', '').lower()
        fraud_detected = any(keyword in description for keyword in self.FRAUD_KEYWORDS)

        if fraud_detected:
            route = "Investigation Queue"
            reasons.append("Potential fraud indicators detected in incident description")
            return route, " | ".join(reasons)

        # Rule 3: Injury claims → Specialist Queue
        claim_type = extracted_fields.get('claim_type', '').lower()
        is_injury = any(injury_type in claim_type for injury_type in self.INJURY_CLAIM_TYPES)

        if is_injury:
            route = "Specialist Queue"
            reasons.append("Claim involves personal injury requiring specialist handling")
            return route, " | ".join(reasons)

        # Rule 4: Low damage → Fast-track
        estimated_damage = extracted_fields.get('estimated_damage', float('inf'))
        if estimated_damage < self.FAST_TRACK_THRESHOLD:
            route = "Fast-Track"
            reasons.append(f"Estimated damage (${estimated_damage:,.2f}) is below fast-track threshold of ${self.FAST_TRACK_THRESHOLD:,}")
            return route, " | ".join(reasons)

        # Default: Standard processing
        route = "Standard Processing"
        reasons.append(f"Claim meets all requirements for standard processing (damage: ${estimated_damage:,.2f})")
        return route, " | ".join(reasons)

    def process_document(self, file_path: str) -> Dict[str, Any]:
        """Main processing function"""
        text = self.extract_from_txt(file_path)

        if not text:
            return {
                "error": "Failed to extract text from document",
                "extractedFields": {},
                "missingFields": [],
                "recommendedRoute": "Error",
                "reasoning": "Could not read document content"
            }

        # Extract fields
        extracted_fields = self.extract_fields(text)

        # Identify missing fields
        missing_fields = self.identify_missing_fields(extracted_fields)

        # Route claim
        recommended_route, reasoning = self.route_claim(extracted_fields, missing_fields)

        # Build result
        result = {
            "extractedFields": extracted_fields,
            "missingFields": missing_fields,
            "recommendedRoute": recommended_route,
            "reasoning": reasoning
        }

        return result

print("ClaimsProcessingAgent class created successfully!")

ClaimsProcessingAgent class created successfully!


## Step 4: Test the Agent with Sample Documents

In [16]:
# Initialize the agent
agent = ClaimsProcessingAgent()

# Test cases
test_cases = [
    ('sample_fnol_1.txt', 'Minor Parking Lot Incident'),
    ('sample_fnol_2.txt', 'Rear-End Collision with Injuries'),
    ('sample_fnol_3.txt', 'Suspicious Hit-and-Run')
]

print("="*80)
print("CLAIMS PROCESSING AGENT - TEST RESULTS")
print("="*80)

for file, description in test_cases:
    print(f"\n{'─'*80}")
    print(f" {description}")
    print(f"{'─'*80}")

    result = agent.process_document(file)

    # Display key extracted fields
    print("\n Key Extracted Fields:")
    key_fields = ['policy_number', 'policyholder_name', 'incident_date', 'estimated_damage', 'claim_type']
    for field in key_fields:
        if field in result['extractedFields']:
            value = result['extractedFields'][field]
            print(f"  • {field.replace('_', ' ').title()}: {value}")

    # Display routing decision
    route_icons = {
        "Fast-Track": " ",
        "Specialist Queue": " ",
        "Investigation Queue": " ",
        "Manual Review": " ",
        "Standard Processing": " "
    }

    route = result['recommendedRoute']
    icon = route_icons.get(route, " ")

    print(f"\n Routing Decision:")
    print(f"  {icon} Route: {route}")
    print(f" Reasoning: {result['reasoning']}")

    if result['missingFields']:
        print(f"Missing Fields: {', '.join(result['missingFields'])}")
    else:
        print(f"  All mandatory fields present")

print(f"\n{'='*80}")
print("All tests completed successfully!")
print("="*80)

CLAIMS PROCESSING AGENT - TEST RESULTS

────────────────────────────────────────────────────────────────────────────────
 Minor Parking Lot Incident
────────────────────────────────────────────────────────────────────────────────

 Key Extracted Fields:
  • Policy Number: AUTO-2024-78901
  • Policyholder Name: Sarah Johnson
  • Incident Date: 02/03/2026
  • Estimated Damage: 1200.0
  • Claim Type: Property Damage

 Routing Decision:
    Route: Fast-Track
 Reasoning: Estimated damage ($1,200.00) is below fast-track threshold of $25,000
  All mandatory fields present

────────────────────────────────────────────────────────────────────────────────
 Rear-End Collision with Injuries
────────────────────────────────────────────────────────────────────────────────

 Key Extracted Fields:
  • Policy Number: AUTO-2024-55432
  • Policyholder Name: Michael Chen
  • Incident Date: 02/01/2026
  • Estimated Damage: 18500.0
  • Claim Type: Personal Injury

 Routing Decision:
    Route: Specialist Qu

## Step 5: View Complete JSON Output

In [17]:
# Process one document and display complete JSON output
result = agent.process_document('sample_fnol_1.txt')

print("Complete JSON Output for Sample FNOL 1:")
print("="*80)
print(json.dumps(result, indent=2))

Complete JSON Output for Sample FNOL 1:
{
  "extractedFields": {
    "policy_number": "AUTO-2024-78901",
    "policyholder_name": "Sarah Johnson",
    "incident_date": "02/03/2026",
    "incident_time": "3:45 PM",
    "incident_location": "123 Main Street, Springfield, IL",
    "incident_description": "Minor fender bender in parking lot. The insured vehicle was backing out of a parking space when it made contact with a stationary shopping cart. Minor dent and scratches on rear bumper. No other vehicles involved. No injuries reported.",
    "claimant_name": "Sarah Johnson",
    "contact_phone": "(555) 234-5678",
    "contact_email": "sarah.johnson@email.com",
    "asset_type": "Vehicle",
    "asset_id": "VIN-1HGBH41JXMN109186",
    "estimated_damage": 1200.0,
    "claim_type": "Property Damage"
  },
  "missingFields": [],
  "recommendedRoute": "Fast-Track",
  "reasoning": "Estimated damage ($1,200.00) is below fast-track threshold of $25,000"
}


## Step 6: Test with Your Own FNOL Document (Optional)

You can upload your own FNOL document and test it here.

In [18]:
# Uncomment and run this cell if you want to upload your own file
# from google.colab import files
# uploaded = files.upload()

# # Process the uploaded file
# for filename in uploaded.keys():
#     print(f"\nProcessing: {filename}")
#     result = agent.process_document(filename)
#     print(json.dumps(result, indent=2))